In [1]:
import xml.etree.ElementTree as ET
import glob

In [2]:
import time
import numpy as np

In [3]:
# we will work with totImages images
totImages = 100

In [4]:
fn = glob.glob("VOC2012/Annotations/*");

In [5]:
images = dict()
cat = set()

In [6]:
start = time.time()

cnt = 0

for f in fn:
    cnt = cnt + 1
    if cnt > totImages:
        break
    
    tree = ET.parse(f)
    root = tree.getroot()

    curFile = ""
    for child in root:
        if child.tag == 'filename':
            curFile = child.text
    images[curFile] = []
    
    for child in root:
        if child.tag == 'object':
            for child2 in child:
                if child2.tag == 'name':
                    images[curFile].append(child2.text)
                    cat.add(child2.text)
                    
print 'Done parsing in ', time.time() - start, " seconds"

Done parsing in  0.0796840190887  seconds


In [7]:
cat = list(cat)
print cat

['sheep', 'horse', 'bicycle', 'bottle', 'cow', 'sofa', 'dog', 'bus', 'cat', 'person', 'train', 'diningtable', 'aeroplane', 'car', 'pottedplant', 'tvmonitor', 'chair', 'bird', 'boat', 'motorbike']


In [8]:
len(cat)

20

In [9]:
catimg = dict()
for curCat in cat:
    catimg[curCat] = []

for img in images:
    for curCat in set(images[img]):
        catimg[curCat].append(img)
        
print catimg['sheep'][:5]

['2007_000925.jpg', '2007_000676.jpg', '2007_000175.jpg', '2007_001416.jpg']


In [10]:
featvec = images.copy()

cnt = 0

for img in images:
    cnt = cnt + 1
    if cnt <= 5:
        print img, images[img]
    
    features = featvec[img]
    
    featvec[img] = [0 for i in range(len(cat))]
    for f in features:
        for j in range(len(cat)):
            if cat[j] == f:
                featvec[img][j] = featvec[img][j] + 1
                break
                
    featvec[img] = np.array(featvec[img])
                
    if cnt <= 5:
        print img, featvec[img]
        print ""

2007_001154.jpg ['sofa', 'pottedplant']
2007_001154.jpg [0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]

2007_000645.jpg ['bird', 'bird']
2007_000645.jpg [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0]

2007_000061.jpg ['boat', 'boat']
2007_000061.jpg [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]

2007_000847.jpg ['diningtable', 'person']
2007_000847.jpg [0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0]

2007_000129.jpg ['bicycle', 'bicycle', 'bicycle', 'person', 'person', 'person']
2007_000129.jpg [0 0 3 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0]



In [27]:
# we will pick the exemplar such that max(nrm(v,v') < nrm(v, exemplar)). 
# taking the exemplar twice(or 3x, 4x) the longest vector should thus suffice
exemplar = [0]
for elem in featvec:
    if np.linalg.norm(featvec[elem]) > np.linalg.norm(exemplar):
        exemplar = featvec[elem][:]
exemplar = exemplar * 3

# compute a dictionary of norms and normdistances
exempDist = featvec.copy()
for elem in featvec:
    exempDist[elem] = np.linalg.norm(featvec[elem] - exemplar)

dist = dict()
for el1 in featvec:
    for el2 in featvec:
        dist[(el1, el2)] = np.linalg.norm(featvec[el1] - featvec[el2])

In [12]:
n = len(images.keys())
m = len(cat)
l = 8
k = 5

In [13]:
from replacementGreedy import replacementGreedy

start = time.time()
rg = replacementGreedy(l, k, featvec, exempDist, dist, catimg)

print 'Ran in ', time.time() - start
print 'Obtained cost', rg[1]

Ran in  0.58779501915
Obtained cost 18.97978888


In [28]:
el0 = featvec.keys()[0]

print 'Distance to exemplar', exempDist[el0]

for other in featvec:
    if other != el0:
        print nrmdist[(other, el0)]

Distance to exemplar 22.4053565024
2.44948974278
2.82842712475
2.82842712475
4.79583152331
1.41421356237
3.16227766017
1.41421356237
2.0
1.0
2.2360679775
1.41421356237
2.44948974278
2.2360679775
2.82842712475
3.60555127546
1.41421356237
1.41421356237
2.2360679775
1.0
2.2360679775
1.0
2.44948974278
2.2360679775
3.60555127546
2.2360679775
2.44948974278
1.73205080757
2.2360679775
3.0
2.82842712475
2.82842712475
6.32455532034
2.2360679775
2.2360679775
1.0
2.2360679775
2.82842712475
1.41421356237
2.82842712475
2.44948974278
2.64575131106
3.60555127546
2.44948974278
1.0
2.2360679775
2.0
2.2360679775
4.24264068712
2.82842712475
2.2360679775
3.74165738677
2.64575131106
2.44948974278
1.0
0.0
3.60555127546
2.82842712475
1.41421356237
5.47722557505
5.83095189485
6.32455532034
4.0
2.2360679775
1.41421356237
4.0
2.2360679775
2.44948974278
1.41421356237
2.2360679775
0.0
2.82842712475
2.44948974278
2.44948974278
2.2360679775
2.2360679775
2.44948974278
1.41421356237
1.41421356237
0.0
2.64575131106
1.4